# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core import Workspace, Experiment, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import os
import pandas as pd

In [3]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="lol_match_predictor_automl")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FST7SA226 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-136540
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-136540


In [6]:
# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
data = pd.read_csv("https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20210126.csv")

# This dataset requires cleaning to only get the features we want.
columns = ['side', 'position', 'result', 'golddiffat10', 'xpdiffat10', 'golddiffat15', 'xpdiffat15']

data = pd.DataFrame(data, columns=columns)

# This dataset has data for both sides in each match. We only need to look at matches from one side, so we take only Blue side data.

data = data.loc[data['side'] == 'Blue']

# This dataset also has data for teams as well as individual players. We only need the data for teams.

data = data[data.position.eq('team')]

# Side no longer needed so we can delete this field.
data.drop(['side'], inplace=True, axis=1, errors='ignore')
# We also no longer need positions.
data.drop(['position'], inplace=True, axis=1, errors='ignore')

# This dataset is equivalent to the output of the above operations.
ds = TabularDatasetFactory.from_delimited_files(path="https://raw.githubusercontent.com/ncharchenko/Azure-ML-Engineer-Nanodegree-Capstone/master/OraclesElixir_automl_data.csv")

# Show data after cleaning.
data.head()

,result,golddiffat10,xpdiffat10,golddiffat15,xpdiffat15
10,1,117.00,-137.00,-825.00,-1665.00
22,0,-2014.00,-2149.00,-5484.00,-4833.00
34,1,682.00,615.00,2566.00,2097.00
46,0,-1579.00,-858.00,-2471.00,-3064.00
58,0,-747.00,-313.00,877.00,205.00


In [27]:
# To show that the AutoML model is taking in the same dataset, show it here.
raw_data = pd.read_csv("https://raw.githubusercontent.com/ncharchenko/Azure-ML-Engineer-Nanodegree-Capstone/master/OraclesElixir_automl_data.csv")
raw_data.head()

,Unnamed: 0,result,golddiffat10,xpdiffat10,golddiffat15,xpdiffat15
0,0,1,117.00,-137.00,-825.00,-1665.00
1,2,0,-2014.00,-2149.00,-5484.00,-4833.00
2,4,1,682.00,615.00,2566.00,2097.00
3,6,0,-1579.00,-858.00,-2471.00,-3064.00
4,8,0,-747.00,-313.00,877.00,205.00


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    enable_onnx_compatible_models=True,
    compute_target=compute_target,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    validation_size=0.2,
    label_column_name="result")

In [9]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/Automated

{'runId': 'AutoML_02b25db5-80fe-4ba2-9c8e-966eb8b4b0c8',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-30T01:18:19.439014Z',
 'endTimeUtc': '2021-01-30T01:57:27.417695Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.2',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"lol_match_predictor_automl","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-136540","workspace_name":"quick-starts-ws-136540","region":"southcentralus","compute_target":"cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.2,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
best_automl, automl_model = remote_run.get_output()
best_automl, onnx_model = remote_run.get_output(return_onnx_model=True)

In [15]:
automl_metrics = best_automl.get_metrics()
print("Best AutoML ID: ", best_automl.id)
print("Accuracy: ", automl_metrics["accuracy"])
print("Model details: ", automl_model)

Best AutoML ID:  AutoML_02b25db5-80fe-4ba2-9c8e-966eb8b4b0c8_28
Accuracy:  0.7498093058733791
Model details:  Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                random_state=None,
                                                                                                reg_alpha=0.05263157894736842,
                                        

In [16]:
import joblib
from azureml.core.model import Model

print(best_automl.get_tags())

joblib.dump(automl_model, filename="automl_model.pkl")
model = remote_run.register_model(model_name=best_automl.properties['model_name'], description="LoL predictor model trained with Azure AutoML.")

{'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}', 'ensembled_iterations': '[10, 26, 19, 16, 6, 12, 20]', 'ensembled_algorithms': "['LightGBM', 'LightGBM', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'GradientBoosting', 'LightGBM', 'LightGBM']", 'ensemble_weights': '[0.09090909090909091, 0.36363636363636365, 0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]', 'best_individual_pipeline_score': '0.7459954233409611', 'best_individual_iteration': '10', '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}


In [17]:
env = best_automl.get_environment()
env.save_to_directory("env.yml", overwrite=True)
entry_script='score.py'
best_automl.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Deploy the model as a Web Service on an Azure Container Instance, just like in Project 2
inference_config = InferenceConfig(entry_script=entry_script, environment=env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)

service = Model.deploy(
    workspace=ws,
    name="lolpredictor",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [20]:
print("Service State: {}".format(service.state))
print("Scoring URI: {}".format(service.scoring_uri))
print("Swagger URI: {}".format(service.swagger_uri))

Service State: Healthy
Scoring URI: http://5b9e625c-3d16-4257-b17c-b9fa9a2b095e.southcentralus.azurecontainer.io/score
Swagger URI: http://5b9e625c-3d16-4257-b17c-b9fa9a2b095e.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [29]:
import requests
import json

scoring_uri = service.scoring_uri

data = {'data':
        [
            {
                'Column1':0,
                'golddiffat10':-500,
                'xpdiffat10':-300,
                'golddiffat15':2000,
                'xpdiffat15':1600
            },
            {
                'Column1':1,
                'golddiffat10':500,
                'xpdiffat10':300,
                'golddiffat15':-2000,
                'xpdiffat15':-1000
            },
        ]
    }

input_data = json.dumps(data)


headers = {'Content-Type': 'application/json'}

response = requests.post(scoring_uri, input_data, headers=headers)
print(response.text)

"{\"result\": [1, 0]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
print(service.get_logs())

2021-01-29T04:18:08,229832500+00:00 - gunicorn/run 
2021-01-29T04:18:08,239220300+00:00 - rsyslog/run 
2021-01-29T04:18:08,236979000+00:00 - iot-server/run 
2021-01-29T04:18:08,314653200+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [ ]:
service.delete()